## Initial Setup
### Import libraries 

In [1]:
import pandas as pd
import os, gc
clean_dataset = 'cleaned'
raw_dataset = 'raw'

### Check if directory exists

In [2]:
if not os.path.isdir(clean_dataset):
    try:
        os.mkdir(clean_dataset)
    except OSError as error:
        print(error)

if not os.path.isdir(raw_dataset):
    try:
        os.mkdir(raw_dataset)
    except OSError as error:
        print(error)

### Utils

In [3]:
def missing_percentage(df):
    return df.isnull().mean().round(4).mul(100).sort_values(ascending=False)

def read_raw(filename):
    return pd.read_csv(os.path.join(raw_dataset, filename))

def read_clean(filename):
    return pd.read_csv(os.path.join(clean_dataset, filename))
    
def write_clean(df:pd.DataFrame, name):
    df.to_csv(os.path.join(clean_dataset, name), index=False)

## Population and density
Source for [Population density](https://covid19.census.gov/datasets/21843f238cbb46b08615fc53e19e0daf_1/explore?showTable=true) and [Population by age and sex](https://www.statsamerica.org/downloads/default.aspx) .

In [26]:
population = read_raw('Population by Age and Sex - US, States, Counties.csv')
population = population[population['Year']==2019].reset_index(drop=True)
population.rename({'IBRC_Geo_ID':'FIPS'}, axis=1, inplace=True)
population.head()

,FIPS,Statefips,Countyfips,Description,Year,Total Population,Population 0-4,Population 5-17,Population 18-24,Population 25-44,Population 45-64,Population 65+,Population Under 18,Population 18-54,Population 55+,Male Population,Female Population
0,0,0,0,U.S.,2019,328239523.0,19576683.0,53462467.0,30219206.0,87599465.0,83323439.0,54058263.0,73039150.0,158693573.0,96506800.0,161692336.0,166637617.0
1,1000,1,0,Alabama,2019,4903185.0,294357.0,793949.0,450224.0,1240535.0,1274283.0,849837.0,1088306.0,2307375.0,1507504.0,2371832.0,2536133.0
2,1001,1,1,"Autauga County, AL",2019,55869.0,3277.0,9688.0,4465.0,14597.0,14918.0,8924.0,12965.0,26714.0,16190.0,27078.0,28691.0
3,1003,1,3,"Baldwin County, AL",2019,223234.0,12039.0,35515.0,15805.0,51839.0,61206.0,46830.0,47554.0,96435.0,79245.0,108396.0,115169.0
4,1005,1,5,"Barbour County, AL",2019,24686.0,1309.0,3773.0,2000.0,6412.0,6331.0,4861.0,5082.0,11494.0,8110.0,13030.0,11627.0


In [27]:
size_density = read_raw('Average_Household_Size_and_Population_Density_-_County.csv')
size_density.rename({'FIPS Code':'FIPS', 'Name':'County'}, axis=1, inplace=True)
size_density = size_density[['FIPS', 'County', 'State', 'Area of Land (square meters)']]

In [28]:
population.drop(['Statefips','Countyfips','Year', 'Description'], axis=1, inplace=True)
population = size_density.merge(population, how='inner', on='FIPS')
population['Population Density'] = population['Total Population'] * 1e6 / population['Area of Land (square meters)']
population.drop('Area of Land (square meters)', axis=1, inplace=True)
write_clean(population, 'Population.csv')

## Google community mobility report
The Region CSVs file is collected from [Google mobility page](https://www.google.com/covid19/mobility/), then US regions are copied from the unziped folder. [Download link](https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip). The date starts from 2020-02-15 till now.

In [29]:
# do the same for complete data and create a summary
county_summaries = []
fix_column_names = {'sub_region_2':'County', 'census_fips_code':'FIPS', 'date':'Date'}
# drop unnecessary columns, these columns are either not necessary or have large missing values
drop_columns = ['place_id', 'country_region_code', 'country_region', 'sub_region_1', 'metro_area','iso_3166_2_code', 
'parks_percent_change_from_baseline', 'transit_stations_percent_change_from_baseline',
'grocery_and_pharmacy_percent_change_from_baseline',
'retail_and_recreation_percent_change_from_baseline'
]

for year in range(2020, 2023):
    filepath = os.path.join(raw_dataset, f'Google mobility report for US/{year}_US_Region_Mobility_Report.csv')
    df = pd.read_csv(filepath)
    df = df.drop(drop_columns, axis=1)

    df = df.rename(fix_column_names, axis=1)
    df = df[~(df['County'].isnull()| df['FIPS'].isnull())]
    df['FIPS'] = df['FIPS'].astype(int)
    df = df[df['FIPS'].isin(population['FIPS'])]
    county_summaries.append(df)

In [30]:
mobility_google = pd.concat(county_summaries, axis=0, sort=False).reset_index(drop=True)
# mobility_google = mobility_google[mobility_google['Date']>=start_date]
missing_percentage(mobility_google)

residential_percent_change_from_baseline    42.35
workplaces_percent_change_from_baseline      1.94
County                                       0.00
FIPS                                         0.00
Date                                         0.00
dtype: float64

In [31]:
mobility_google = mobility_google[['FIPS', 'Date', 'workplaces_percent_change_from_baseline']]
write_clean(mobility_google, 'Mobility google.csv')

## Trips by Distance data
Collected from [U.S. Department of Transportation Bureau of Transportation Statistics](https://data.bts.gov/Research-and-Statistics/Trips-by-Distance/w96p-f2qv). [Download link](https://data.bts.gov/api/views/w96p-f2qv/rows.csv?accessType=DOWNLOAD). From 2019/01/01 till 2022/02/05, updated regularly.

In [43]:
mobility_bts = read_raw("Trips_by_distance.csv")
mobility_bts.columns

Index(['Level', 'Date', 'State FIPS', 'State Postal Code', 'County FIPS',
       'County Name', 'Population Staying at Home',
       'Population Not Staying at Home', 'Number of Trips',
       'Number of Trips <1', 'Number of Trips 1-3', 'Number of Trips 3-5',
       'Number of Trips 5-10', 'Number of Trips 10-25',
       'Number of Trips 25-50', 'Number of Trips 50-100',
       'Number of Trips 100-250', 'Number of Trips 250-500',
       'Number of Trips >=500', 'Row ID', 'Week', 'Month'],
      dtype='object')

In [44]:
print(mobility_bts.Date.min(), mobility_bts.Date.max())

mobility_bts = mobility_bts.drop(['State FIPS', 'State Postal Code',
'Row ID', 'Week', 'Month'], axis=1)
mobility_bts = mobility_bts[mobility_bts['Level']=='County'].reset_index(drop=True)

# replace datetime format 'year/month/day' with 'year-month-day' to be consistent with others
mobility_bts['Date'] = pd.to_datetime(mobility_bts['Date'])

# reducing file size
mobility_bts = mobility_bts[mobility_bts['Date']>pd.to_datetime('2020-02-01')]

missing_percentage_county = missing_percentage(mobility_bts)
# missing_percentage_county[missing_percentage_county>0]

2019/01/01 2022/02/05


In [45]:
# mobility_bts['Total Population'] = mobility_bts['Population Staying at Home']+mobility_bts['Population Not Staying at Home']
# mobility_bts['Ratio of Population Not Staying at Home'] = mobility_bts['Population Not Staying at Home']/ mobility_bts['Total Population']
# mobility_bts['Number of Trips Per Person'] = mobility_bts['Number of Trips'] / mobility_bts['Total Population']

mobility_bts.rename({'County FIPS':'FIPS', 'County Name':'County'}, axis=1, inplace=True)
mobility_bts['FIPS'] = mobility_bts['FIPS'].astype(int)

In [46]:
selected_columns = ['FIPS', 'Date', 'Number of Trips','Population Not Staying at Home']

# fill the rest 0.49% missing values with values from previous non-null row
# missing_percentage(mobility_bts)
mobility_bts = mobility_bts[selected_columns].fillna(method='ffill')
write_clean(mobility_bts, "Mobility bts.csv")

## Covid cases
Collected cumulative covid cases from [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). It is then converted to daily cases, neg daily cases are converted to zero, then dumped in the cleaned folder.

In [15]:
df = read_raw('covid_confirmed_usafacts.csv')
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-03-30,2022-03-31,2022-04-01,2022-04-02,2022-04-03,2022-04-04,2022-04-05,2022-04-06,2022-04-07,2022-04-08
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,15621,15655,15659,15659,15659,15689,15692,15724,15742,15744
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,55408,55414,55420,55420,55420,55431,55449,55456,55468,55489
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,5656,5656,5656,5656,5656,5656,5656,5657,5658,5658
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,6418,6418,6417,6417,6417,6419,6419,6420,6421,6422


In [16]:
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)

# drop invalid rows
df = df[~df['FIPS'].isna()]
df = df[df['FIPS']>0]
df = df.fillna(0)

# convert cumulative cases to daily 
df = df.T
head = df.iloc[0]
df = df.iloc[1:]
df.columns = head
df = df.diff()
df = df.T.reset_index()

In [17]:
df = df.melt(
    id_vars= ['FIPS'],
    var_name='Date', value_name='Cases'
).reset_index(drop=True)
df = df.fillna(0)

# some days had old covid cases fixed by adding neg values
df.loc[df['Cases']<0, 'Cases'] = 0

write_clean(df, 'Cases.csv')

df.head()

,FIPS,Date,Cases
0,1001,2020-01-22,0.0
1,1003,2020-01-22,0.0
2,1005,2020-01-22,0.0
3,1007,2020-01-22,0.0
4,1009,2020-01-22,0.0


## Deaths
Collected cumulative covid deaths from [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). It is then converted to daily deaths, neg daily deaths are converted to zero, then dumped in the cleaned folder.

In [12]:
df = read_raw('covid_deaths_usafacts.csv')
df.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-04-01,2022-04-02,2022-04-03,2022-04-04,2022-04-05,2022-04-06,2022-04-07,2022-04-08,2022-04-09,2022-04-10
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,211,211,211,213,213,213,213,213,213,213
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,675,675,675,675,675,676,676,677,677,677
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,97,97,97,97,97,98,98,98,98,98
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,101,101,101,101,101,101,101,101,101,101


In [13]:
df.rename({'countyFIPS':'FIPS'}, axis=1, inplace=True)
df.drop(columns=['County Name', 'State', 'StateFIPS'], inplace=True)

# drop invalid rows
df = df[~df['FIPS'].isna()]
df = df[df['FIPS']>0]
df = df.fillna(0)

# convert cumulative cases to daily 
df = df.T
head = df.iloc[0]
df = df.iloc[1:]
df.columns = head
df = df.diff()
df = df.T.reset_index()

In [14]:
df = df.melt(
    id_vars= ['FIPS'],
    var_name='Date', value_name='Deaths'
).reset_index(drop=True)

df = df.fillna(0)
# some days had old covid deaths fixed by adding neg values
df.loc[df['Deaths']<0, 'Deaths'] = 0

write_clean(df, 'Deaths.csv')

df.head()

,FIPS,Date,Deaths
0,1001,2020-01-22,0.0
1,1003,2020-01-22,0.0
2,1005,2020-01-22,0.0
3,1007,2020-01-22,0.0
4,1009,2020-01-22,0.0


## Vaccinations

In [33]:
vaccination = read_raw('COVID-19_Vaccinations_in_the_United_States_County.csv')
print(vaccination.columns)

Index(['Date', 'FIPS', 'MMWR_week', 'Recip_County', 'Recip_State',
       'Completeness_pct', 'Administered_Dose1_Recip',
       'Administered_Dose1_Pop_Pct', 'Administered_Dose1_Recip_5Plus',
       'Administered_Dose1_Recip_5PlusPop_Pct',
       'Administered_Dose1_Recip_12Plus',
       'Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18Plus',
       'Administered_Dose1_Recip_18PlusPop_Pct',
       'Administered_Dose1_Recip_65Plus',
       'Administered_Dose1_Recip_65PlusPop_Pct', 'Series_Complete_Yes',
       'Series_Complete_Pop_Pct', 'Series_Complete_5Plus',
       'Series_Complete_5PlusPop_Pct', 'Series_Complete_5to17',
       'Series_Complete_5to17Pop_Pct', 'Series_Complete_12Plus',
       'Series_Complete_12PlusPop_Pct', 'Series_Complete_18Plus',
       'Series_Complete_18PlusPop_Pct', 'Series_Complete_65Plus',
       'Series_Complete_65PlusPop_Pct', 'Booster_Doses',
       'Booster_Doses_Vax_Pct', 'Booster_Doses_12Plus',
       'Booster_Doses_12Plus_V

In [34]:
vaccination = vaccination[vaccination['FIPS']!='UNK']
vaccination['FIPS'] = vaccination['FIPS'].astype(int)

vaccination = vaccination[['Date', 'FIPS', 'Administered_Dose1_Recip','Administered_Dose1_Pop_Pct',
'Administered_Dose1_Recip_12Plus', 'Administered_Dose1_Recip_12PlusPop_Pct',
'Series_Complete_Yes', 'Series_Complete_Pop_Pct', 'Booster_Doses', 'Booster_Doses_Vax_Pct']]

vaccination['Date'] = pd.to_datetime(vaccination['Date'])

selected_columns = ['Date', 'FIPS', 'Administered_Dose1_Recip','Series_Complete_Yes']
write_clean(vaccination[selected_columns], "Vaccination.csv")

## Social Vulnerability Index

In [32]:
svi = read_raw('CDC_Social_Vulnerability_Index_2018_-_USA.csv')
selected_columns = ['FIPS'] + [col for col in svi.columns if 'RPL_THEME' in col]
write_clean(svi[selected_columns], 'Social vulnerability index.csv')

svi[selected_columns].head()

,FIPS,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES
0,1001,0.3631,0.5810,0.5947,0.3741,0.4354
1,1003,0.2232,0.1990,0.4358,0.3359,0.2162
2,1005,0.9780,0.9153,0.8558,0.9889,0.9959
3,1007,0.7694,0.1203,0.4323,0.7189,0.6003
4,1009,0.6143,0.3187,0.5915,0.1741,0.4242


## County Health Ranking

In [33]:
# https://github.com/BindiChen/machine-learning/blob/main/data-analysis/031-pandas-multiIndex/multiindex-selection.ipynb
health_ranking = pd.read_excel(os.path.join(raw_dataset, '2021 County Health Rankings Data - v1.xlsx'), sheet_name=None, header=[0, 1])
ranked_measure = health_ranking['Ranked Measure Data']
ranked_measure.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Premature death  \
                FIPS              State             County      Unreliable   
0               1000            Alabama                NaN             NaN   
1               1001            Alabama            Autauga             NaN   
2               1003            Alabama            Baldwin             NaN   
3               1005            Alabama            Barbour             NaN   
4               1007            Alabama               Bibb             NaN   

                                                                          \
    Deaths Years of Potential Life Lost Rate  95% CI - Low 95% CI - High   
0  82249.0                       9819.887431   9718.253592   9921.521270   
1    787.0                       7830.053484   6997.660326   8662.446641   
2   3147.0                       7680.477270   7236.921021   8124.033519   
3    515.0                      11476.629416   9907.793139  13045.465692   
4    476.0                      12172.562382  10506.324118  13838.800647   

                             ...                 Driving alone to work  \
  Quartile YPLL Rate (AIAN)  ... % Drive Alone (Hispanic) 95% CI - Low   
0      NaN      5145.190874  ...                             75.976683   
1      1.0              NaN  ...                                   NaN   
2      1.0              NaN  ...                             62.064858   
3      3.0              NaN  ...                                   NaN   
4      4.0              NaN  ...                                   NaN   

                                                                \
  % Drive Alone (Hispanic) 95% CI - High % Drive Alone (White)   
0                              78.432035             87.006161   
1                                    NaN             82.121685   
2                              82.521483             82.207333   
3                                    NaN             86.445199   
4                                    NaN                   NaN   

                                                                          \
  % Drive Alone (White) 95% CI - Low % Drive Alone (White) 95% CI - High   
0                          86.725587                           87.286735   
1                          77.711383                           86.531987   
2                          80.188607                           84.226059   
3                          81.622920                           91.267478   
4                                NaN                                 NaN   

  Long commute - driving alone                                             \
     # Workers who Drive Alone % Long Commute - Drives Alone 95% CI - Low   
0                    2073072.0                          34.6    34.170402   
1                      24635.0                          38.3    34.393441   
2                      93141.0                          40.4    38.025974   
3                       8231.0                          30.9    25.849153   
4                       8167.0                          52.0    43.804666   

                          
  95% CI - High Quartile  
0     35.029598      NaN  
1     42.206559      2.0  
2     42.774026      3.0  
3     35.950847      2.0  
4     60.195334      4.0  

[5 rows x 249 columns]

In [34]:
ranked_measure = ranked_measure.loc[:, 
    [
        ('Unnamed: 0_level_0','FIPS'),
        ('Unnamed: 2_level_0','County'),
        # ('Premature death','Years of Potential Life Lost Rate'),
        ('Poor or fair health', '% Fair or Poor Health'),
        ('Adult smoking', '% Smokers'),
        ('Adult obesity', '% Adults with Obesity'),
        ('Primary care physicians', 'Primary Care Physicians Rate'),
        ('Flu vaccinations', '% Vaccinated'),
        ('Unemployment', '% Unemployed'),
        ('Air pollution - particulate matter', 'Average Daily PM2.5'),
        ('Severe housing problems', '% Severe Housing Problems')
    ]
]
ranked_measure.columns = ranked_measure.columns.get_level_values(1)
ranked_measure = ranked_measure[~ranked_measure['County'].isna()].drop('County', axis=1)
missing_percentage(ranked_measure)

Primary Care Physicians Rate    4.77
Average Daily PM2.5             0.83
% Vaccinated                    0.57
% Unemployed                    0.03
FIPS                            0.00
% Fair or Poor Health           0.00
% Smokers                       0.00
% Adults with Obesity           0.00
% Severe Housing Problems       0.00
dtype: float64

In [35]:
ranked_measure.rename({'% Vaccinated': '% Flu Vaccinated'}, axis=1, inplace=True)
write_clean(ranked_measure, 'Health rank measure.csv')

## Test data

In [36]:
tests = read_raw('covid_testing_cdc.csv')
tests = tests.fillna(0).melt(
    id_vars=['fips_code'], 
    var_name="Date", 
    value_name="Tests"
).reset_index(drop=True)
tests.rename({'fips_code':'FIPS'}, axis=1, inplace=True)
# tests = tests[tests['Date']>=start_date]

tests = tests[['Date', 'FIPS', 'Tests']].pivot_table(
    values='Tests',
    index=['FIPS'],
    columns='Date'
).reset_index().rename_axis(None, axis=1)

write_clean(tests, 'Testing.csv')